# Map Gaze to video frames

We will download a different neon recording containing a vieo file this time.

In [1]:
import sys
import numpy as np
from pyneon import NeonDataset, NeonRecording
import pandas as pd

# define path from root directory of repository
recording_dir = "../../data/OpticalFlow/data"

recording = NeonRecording(recording_dir)

print(recording)


Recording ID: 9265f7c1-e3ce-4108-9dd1-639305591f79
Wearer ID: 990a3de0-5a1a-4f07-bf9b-d82369213dbf
Wearer name: Qian
Recording start time: 2024-07-24 16:31:22.223000
Recording duration: 96.226 s
                 exist              filename                                              path
3d_eye_states     True     3d_eye_states.csv     ..\..\data\OpticalFlow\data\3d_eye_states.csv
blinks            True            blinks.csv            ..\..\data\OpticalFlow\data\blinks.csv
events            True            events.csv            ..\..\data\OpticalFlow\data\events.csv
fixations         True         fixations.csv         ..\..\data\OpticalFlow\data\fixations.csv
gaze              True              gaze.csv              ..\..\data\OpticalFlow\data\gaze.csv
imu               True               imu.csv               ..\..\data\OpticalFlow\data\imu.csv
labels            True            labels.csv            ..\..\data\OpticalFlow\data\labels.csv
saccades          True          saccades.csv

We can see that a scene_video object exists now. The issue with neon recordings is that event data is not naturally synched to the video. Therefore, receiving an overlay of the current gaze or alternatively, the tracking of past fixations, requires further steps.

We will firstly perform a mapping of the current gaze to the video. To do this, we pick the closest timestamp within gaze.csv for every video timestamp in world_timestamps.

In [2]:
recording.map_gaze_to_video()

,timestamp [ns],time [s],gaze x [px],gaze y [px],worn,fixation id,blink id,azimuth [deg],elevation [deg],fixation status
0,1721831484673000000,0.000000,760.726011,534.588942,True,1,<NA>,-3.529437,4.794493,start
1,1721831484723000000,0.050000,760.598421,534.750212,True,1,<NA>,-3.537660,4.784086,during
2,1721831484773000000,0.100000,761.261772,535.061108,True,1,<NA>,-3.494775,4.764158,during
3,1721831484823000000,0.150000,759.370052,544.784969,True,1,<NA>,-3.615135,4.137944,during
4,1721831484873000000,0.200000,755.954625,550.476740,True,1,<NA>,-3.834420,3.771180,during
...,...,...,...,...,...,...,...,...,...,...
2799,1721831578297311111,93.624311,844.043876,841.339927,True,219,<NA>,1.886802,-14.938702,during
2800,1721831578330655555,93.657656,838.680547,853.759814,True,219,<NA>,1.537392,-15.735576,during
2801,1721831578364000000,93.691000,834.099360,854.897830,True,219,<NA>,1.235477,-15.809222,during
2802,1721831578397344444,93.724344,839.294234,848.852753,True,219,<NA>,1.576386,-15.421058,during


In calling the method, we can further specify where a pickled output is saved. Why a pickle you may ask...

In [3]:


recording.estimate_scanpath()



KeyError: 'fixation id'

In [7]:
recording.estimated_scanpath.to_pickle("estimated_scanpath.pkl")
recording.estimated_scanpath.to_csv("estimated_scanpath.csv")

If we print the output, we can see that our mapped gaze data is a dataframe OF dataframes. For every frame, we have a dataframe of fixations with their corresponding fixation_id, x, y-values and status.

This is motivated by the possibility of multiple fixations being present in any given world frame. Even while we have an active fixation, we can still track past fixations through an optical flow algorithm. Eventually, we have multiple fixations, at ifferent locations and crucially with different status, within the same frame.

The standard choice for an optical flow algorithm is Lucas-Kanade, as we mostly care for the movement of sparse points within the scene. Of course, it would be possible to alternatively compute the entire visual flow field and compute the movement of points accordingly.

Tracking fixations with optical flow takes a bit of time, so make a cup of coffee.

In [2]:
import sys
import numpy as np
from pyneon import NeonDataset, NeonRecording
import pandas as pd

# define path from root directory of repository
recording_dir = "../../data/OpticalFlow/data"

recording = NeonRecording(recording_dir)

print(recording)
# import from pkl
recording.estimated_scanpath = pd.read_pickle("estimated_scanpath.pkl")

recording.overlay_scanpath_on_video('test.mp4', show_video=True)


Recording ID: 9265f7c1-e3ce-4108-9dd1-639305591f79
Wearer ID: 990a3de0-5a1a-4f07-bf9b-d82369213dbf
Wearer name: Qian
Recording start time: 2024-07-24 16:31:22.223000
Recording duration: 96.226 s
                 exist              filename                                              path
3d_eye_states     True     3d_eye_states.csv     ..\..\data\OpticalFlow\data\3d_eye_states.csv
blinks            True            blinks.csv            ..\..\data\OpticalFlow\data\blinks.csv
events            True            events.csv            ..\..\data\OpticalFlow\data\events.csv
fixations         True         fixations.csv         ..\..\data\OpticalFlow\data\fixations.csv
gaze              True              gaze.csv              ..\..\data\OpticalFlow\data\gaze.csv
imu               True               imu.csv               ..\..\data\OpticalFlow\data\imu.csv
labels            True            labels.csv            ..\..\data\OpticalFlow\data\labels.csv
saccades          True          saccades.csv

We can now print a single 'fixations'-cell from the outer dictionary:

In [2]:
import sys
import numpy as np
from pyneon import NeonDataset, NeonRecording
import pandas as pd

# define path from root directory of repository
recording_dir = "../../data/OpticalFlow/data"

recording = NeonRecording(recording_dir)

print(recording.gaze.data.head())

print(type(recording.video.ts))
recording.map_gaze_to_video()
print(recording.mapped_gaze)

        timestamp [ns]  gaze x [px]  gaze y [px]  worn  fixation id  blink id  \
0  1721831484624880332      765.012      537.417  True            1      <NA>   
1  1721831484629880332      763.474      533.085  True            1      <NA>   
2  1721831484634880332      763.745      533.122  True            1      <NA>   
3  1721831484639880332      763.249      533.265  True            1      <NA>   
4  1721831484644879332      761.531      537.360  True            1      <NA>   

   azimuth [deg]  elevation [deg]  time [s]  
0      -3.252191         4.612959  0.000000  
1      -3.352273         4.891627  0.005000  
2      -3.334809         4.889307  0.010000  
3      -3.366797         4.880000  0.015000  
4      -3.476923         4.616209  0.019999  
<class 'numpy.ndarray'>
           timestamp [ns]   time [s]  gaze x [px]  gaze y [px]  worn  \
0     1721831482223000000   0.000000          NaN          NaN   NaN   
1     1721831482273000000   0.050000          NaN          NaN   NaN 

In [3]:
recording.estimate_scanpath()

,time,fixations
0,0.000000,fixation id x y fixation status 0 ...
1,0.050000,fixation id x y fixation status 1 ...
2,0.100000,fixation id x y fixation status 2 ...
3,0.150000,fixation id x y fixation status 3 ...
4,0.200000,fixation id x y fixation status 4 ...
...,...,...
2848,96.074311,fixation id x y fixation...
2849,96.107656,fixation id x y fixation...
2850,96.141000,fixation id x y fixation...
2851,96.174344,fixation id x y fixation...


In [4]:
recording.estimated_scanpath.to_pickle("estimated_scanpath.pkl")
recording.estimated_scanpath.to_csv("estimated_scanpath.csv")

In [5]:
recording.overlay_scanpath_on_video('test.mp4', show_video=True)

As we can see, we now have multiple fixations that are present within the same frame. Fixation 10 is currently active, indicating that Neon interprets it as being maintained. Frames 8, 7, 5, 4 are tracked within the scene, whereas 9 and 6 have already been dropped.

The logic behin the status is the following. Whenever a new frame has been detected, it gets flagged as "onset". For all future frames during which it is monitored, the flag changes to "active". The first frame in which it is no longer active, it gets flagged as "offset", passing it to the optical flow algorithm. While the algorithm manages to track it, the flag changes to "tracked". When the tracking is lost, the flag changes to "lost" after which it is dropped from future tracking.

Calling .overlay_fixations_on_video allows us to create a video with overlaid current and past fixations. The blue dot represents the current gaze location, whereas green dots represent tracked fixations. A red central dot indicates that the current moment is not recognised as a fixation (but rather a saccade or blink)

In [9]:
recording.overlay_fixations_on_video("../../data/OpticalFlow/overlayed.mp4")